In [ ]:
import scipy.io
import os
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftshift
from scipy.signal import hilbert
from scipy.signal.windows import tukey 
import numpy as np
import helper_functions as hf
import vectorised_hf as vhf

# Load just the waveform from the data file
data = scipy.io.loadmat('data/ultrasound_reflection_data.mat', variable_names=['__header__', '__version__', '__globals__', 'elementPositions', 'samplingFrequency', 'soundSpeed','waveform', 'rcvData'])
waveform = data['waveform']
print(type(data))
print (data.keys())


In [ ]:
# Extract the numpy arrays from the dictionary using their variable names
samplingFrequency = data['samplingFrequency']
elementPositions = data['elementPositions']
soundSpeed = data['soundSpeed']
rcvData = data['rcvData']
Ntx, Nrx, Nt = rcvData.shape

In [ ]:
rawData2d = rcvData.reshape(Ntx * Nrx, Nt)
envelope2d = vhf.preProcessedDataVectorised(rawData2d, alpha=0.1, noise_length=300)


In [ ]:
plt.plot (envelope2d[1,:])

In [ ]:

# plt.plot(envelope)
envelope_3d = vhf.reshapeTo3D(envelope2d)

In [ ]:
# Test the create Imaging Grid function with dx = 0.0003 and Lx = 0.24
dx = 0.002 
Lx = 0.24
X, Y = hf.createImagingGrid(dx, Lx)
Npx = X.size
print(Npx)

In [ ]:
# Precompute the distance map here [m]

# Convert this to a delay map [s]

# Convert this to a sample index travel time map [samples]

In [ ]:
# Initialise accumulator
accumulator = 0

In [ ]:
accumulated_values = np.zeros(X.shape)

In [ ]:
total_pixels = X.shape[0] * X.shape[1]
current_pixel = 0

for pixel_row in range(X.shape[0]):
    for pixel_col in range(X.shape[1]):
        accumulator = 0
        Xp = X[pixel_row, pixel_col]
        Yp = Y[pixel_row, pixel_col]
        # Loop over all transmitters and receivers
        for Tx in [0]: 
            for Rx in range(256):  
                x = hf.accumulate_signal(Tx, Rx, Xp, Yp, elementPositions, soundSpeed, samplingFrequency, rcvData) # Not sure how to include processed data, 
                                                                             # since the 3d matrix is embedded back into the accumulate function, instead of the pre processed vectorisation function
        accumulated_values[pixel_row, pixel_col] = accumulator
        # Update progress
        current_pixel += 1
        percentage_completed = (current_pixel / total_pixels) * 100
        print(f"{percentage_completed:.2f}%", end='\r') # prints to 2dp, with a return

In [ ]:
# Plot X and Y using plt.imshow
plt.imshow(accumulated_values)